In [1]:
import requests
import pandas as pd
from requests_html import HTMLSession
import datetime

In [2]:
sector_index = {
    'NIFTY 50' : 'NIFTY%2050',
    'NIFTY NEXT 50' : 'NIFTY%20NEXT%2050'
}

In [3]:
# Creating a session object
session = HTMLSession()

# Make a GET request to the website to obtain the initial cookie
response = session.get('https://www.nseindia.com')

# Extract the cookie value from the response
cookie_value = session.cookies.get('cookie_name')

headers = {
    'Referer': 'https://www.nseindia.com',
    'Cookie': f'cookie_name={cookie_value}',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [20]:
# Creating a empty dataframe
combined_df = pd.DataFrame()

for sector_name,sector_index_value in sector_index.items():
    
    sector_live_data_url = f'https://www.nseindia.com/api/equity-stockIndices?index={sector_index_value}'

    response = session.get(sector_live_data_url)
    
    df = pd.DataFrame(response.json()['data'])[['priority','symbol']]

    # Removing the Sector Name Row from the dataframe
    df = df[df['priority'] != 1]

    # Removing the priority column from the dataframe
    df = df.drop(['priority'],axis=1)

    # Creating new column and assigning the name of the sector
    df['SECTOR'] = sector_name

    # Renaming the column name
    df = df.rename(columns={'symbol':'SYMBOL'})

    combined_df = pd.concat([combined_df, df], ignore_index=True)

In [21]:
combined_df.head()

,SYMBOL,SECTOR
0,GRASIM,NIFTY 50
1,ITC,NIFTY 50
2,BPCL,NIFTY 50
3,BAJFINANCE,NIFTY 50
4,RELIANCE,NIFTY 50


In [22]:
security_full_bhavdata_url = 'https://archives.nseindia.com/products/content/sec_bhavdata_full_30062023.csv'

In [23]:
response = session.get(security_full_bhavdata_url)
response

<Response [200]>

In [24]:
bhavdata_filename = response.url[security_full_bhavdata_url.rfind('/')+1:]
bhavdata_file_path = 'data/sec_bhavdata_full/' + bhavdata_filename
bhavdata_file_path

'data/sec_bhavdata_full/sec_bhavdata_full_30062023.csv'

In [25]:
# with open(bhavdata_filename, 'wb') as f:
#     f.write(response.content)

In [5]:
trading_holiday_url = 'https://www.nseindia.com/api/holiday-master?type=trading'
response = session.get(trading_holiday_url)
response

<Response [200]>

In [6]:
data = response.json().get('CM',[])
data
df = pd.DataFrame(data,columns=['tradingDate', 'weekDay'])
df = df.rename(columns ={'tradingDate':'HolidayDate'})
df.head()

,HolidayDate,weekDay
0,26-Jan-2023,Thursday
1,18-Feb-2023,Saturday
2,07-Mar-2023,Tuesday
3,30-Mar-2023,Thursday
4,04-Apr-2023,Tuesday


In [28]:
df['HolidayDate'] = df['HolidayDate'].str.strip()
# Convert column to datetime
df['HolidayDate'] = pd.to_datetime(df['HolidayDate'], format='%d-%b-%Y')
# df['HolidayDate'] = df['HolidayDate'].dt.strftime('%d-%m-%Y')
df.dtypes

HolidayDate    datetime64[ns]
weekDay                object
dtype: object

In [29]:
today = datetime.datetime.now().strftime('%d-%b-%Y')
today

'04-Jul-2023'

In [30]:
current_date = datetime.datetime.now()
current_date

datetime.datetime(2023, 7, 4, 6, 33, 27, 873002)

In [31]:
one_day = datetime.timedelta(days=1)
one_day

datetime.timedelta(days=1)

In [32]:
previous_day = current_date - one_day

In [33]:
previous_day

datetime.datetime(2023, 7, 3, 6, 33, 27, 873002)

In [38]:
previous_trading_day = pd.to_datetime('02-Jul-2023')
previous_trading_day = previous_trading_day.strftime("%d-%m-%y").replace("-", "")
# previous_trading_day.replace("-", "")
previous_trading_day

'020723'